# Run the wavelength calibration notebook first!

In [1]:
import os
os.environ['OPENTSDB_PYTHON_METRICS_TEST_MODE'] = 'True'

In [2]:
from banzai.calibrations import make_master_calibrations
import importlib.resources
import requests
from banzai_floyds import settings
from banzai.utils.stage_utils import run_pipeline_stages
from banzai.utils.fits_utils import download_from_s3
import numpy as np
from astropy.io import ascii

In [3]:
os.environ['DB_ADDRESS'] = 'sqlite:///test_data/test.db'
settings.processed_path= os.path.join(os.getcwd(), 'test_data')
settings.fpack=True
settings.db_address = os.environ['DB_ADDRESS']
settings.RAW_DATA_FRAME_URL = f'https://archive-api.lco.global/frames'

In [4]:
# set up the context object.
import banzai.main
context = banzai.main.parse_args(settings, parse_system_args=False)

In [5]:
test_data = ascii.read(os.path.join(importlib.resources.files('banzai_floyds.tests'), 'data/test_data.dat'))
for row in test_data:
    if 'w00.fits' in row['filename']:
        run_pipeline_stages([{'filename': row['filename'], 'RLEVEL': 0, 'frameid': row['frameid']}], context)

2024-01-16 11:57:45.713     INFO:      fits_utils: Downloading file coj2m002-en12-20200813-0016-w00.fits.fz from archive. ID: 33851505. | {"filename": "coj2m002-en12-20200813-0016-w00.fits.fz", "attempt_number": 1, "processName": "MainProcess"}
2024-01-16 11:57:48.045     INFO:          stages: Running banzai.stages.OverscanSubtractor | {"filename": "coj2m002-en12-20200813-0016-w00.fits.fz", "site": "coj", "instrument": "floyds02", "epoch": "20200813", "request_num": "2207712", "obstype": "LAMPFLAT", "filter": "air", "processName": "MainProcess"}
2024-01-16 11:57:48.047     INFO:          stages: Running banzai.stages.Trimmer | {"filename": "coj2m002-en12-20200813-0016-w00.fits.fz", "site": "coj", "instrument": "floyds02", "epoch": "20200813", "request_num": "2207712", "obstype": "LAMPFLAT", "filter": "air", "processName": "MainProcess"}
2024-01-16 11:57:48.048     INFO:            trim: Trimming image | {"filename": "coj2m002-en12-20200813-0016-w00.fits.fz", "site": "coj", "instrument

In [6]:
observations = {'request': {'configuration': {'LAMPFLAT': {'instrument_configs': {'exposure_count': 1}}}}}
intruments = [banzai.dbs.get_instruments_at_site(site, context.db_address)[0] for site in ['ogg', 'coj']]
for instrument in intruments:
    make_master_calibrations(instrument, 'LAMPFLAT', '2015-01-01', '2025-01-01', context)

2024-01-16 11:57:57.398     INFO:    calibrations: Making master frames | {"type": "2m0-FLOYDS-SciCam", "site": "ogg", "camera": "en06", "obstype": "LAMPFLAT", "min_date": "2015-01-01", "max_date": "2025-01-01", "processName": "MainProcess"}
2024-01-16 11:57:57.592     INFO:          stages: Running banzai.stages.FringeMaker | {"filename": "ogg2m001-en06-20200822-0008-w91.fits.fz", "site": "ogg", "instrument": "floyds01", "epoch": "20200822", "request_num": "2217414", "obstype": "LAMPFLAT", "filter": "air", "processName": "MainProcess"}
2024-01-16 11:58:21.911  WARNING:        warnings: /Users/cmccully/miniconda3/envs/banzai-floyds/lib/python3.10/site-packages/astropy/io/fits/card.py:1038: VerifyWarning: Card is too long, comment will be truncated.
  warnings.warn(

2024-01-16 11:58:22.024     INFO:    calibrations: Finished | {"processName": "MainProcess"}
2024-01-16 11:58:22.025     INFO:    calibrations: Making master frames | {"type": "2m0-FLOYDS-SciCam", "site": "coj", "camera": "